In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
path = ''

In [3]:
# !git clone --branch u7 https://github.com/WongKinYiu/yolov7.git  # original repo
!unzip "{path}"yolov7_seg.zip

Archive:  /content/drive/Shareddrives/P&I/Data & ML/PV&rooftop_project/yolov7_seg.zip
   creating: yolov7_seg/
   creating: yolov7_seg/det/
  inflating: yolov7_seg/.DS_Store    
  inflating: __MACOSX/yolov7_seg/._.DS_Store  
   creating: yolov7_seg/cls/
  inflating: yolov7_seg/README.md    
   creating: yolov7_seg/seg/
   creating: yolov7_seg/.git/
   creating: yolov7_seg/det/classify/
  inflating: yolov7_seg/det/LICENSE  
  inflating: yolov7_seg/det/requirements.txt  
  inflating: yolov7_seg/det/val.py   
   creating: yolov7_seg/det/utils/
   creating: yolov7_seg/det/models/
  inflating: yolov7_seg/det/export.py  
  inflating: yolov7_seg/det/README.md  
  inflating: yolov7_seg/det/train.py  
  inflating: yolov7_seg/det/setup.cfg  
  inflating: yolov7_seg/det/hubconf.py  
   creating: yolov7_seg/det/data/
  inflating: yolov7_seg/det/detect.py  
  inflating: yolov7_seg/cls/classifier.py  
  inflating: yolov7_seg/cls/LICENSE  
  inflating: yolov7_seg/cls/requirements.txt  
  inflating: y

In [4]:
%cd yolov7_seg/seg

/content/yolov7_seg/seg


In [ ]:
!pip install -r requirements.txt

In [ ]:
import torch
from IPython.display import Image
import os
import random
import shutil
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET
from xml.dom import minidom
from tqdm import tqdm
from PIL import Image, ImageDraw
import numpy as np
import matplotlib.pyplot as plt

random.seed(108)

In [ ]:
!unzip "{path}"yolo_segmentation_data.zip -d "/content/"

In [ ]:
# create folders for the data after splitting
!mkdir split_data
!mkdir split_data/images
!mkdir split_data/labels
!mkdir split_data/images/train
!mkdir split_data/labels/train
!mkdir split_data/images/val
!mkdir split_data/labels/val
!mkdir split_data/images/test
!mkdir split_data/labels/test

In [ ]:
ims_path = '/content/yolo_data/images/'
annot_path = '/content/yolo_data/labels/'
# read images and annotations
images = [os.path.join(ims_path, x) for x in os.listdir(ims_path)]
annotations = [os.path.join(annot_path, x) for x in os.listdir(annot_path) if x[-3:] == "txt"]

In [ ]:
images.sort()
annotations.sort()

# split the dataset to train-validation-test
train_images, val_images, train_annotations, val_annotations = train_test_split(images, annotations, test_size = 0.2, random_state = 1)
val_images, test_images, val_annotations, test_annotations = train_test_split(val_images, val_annotations, test_size = 0.5, random_state = 1)

In [ ]:
def move_files_to_folder(list_of_files, destination_folder):
    for f in list_of_files:
        try:
            shutil.move(f, destination_folder)
        except:
            print(f)
            assert False

move_files_to_folder(train_images, 'split_data/images/train')
move_files_to_folder(val_images, 'split_data/images/val/')
move_files_to_folder(test_images, 'split_data/images/test/')
move_files_to_folder(train_annotations, 'split_data/labels/train/')
move_files_to_folder(val_annotations, 'split_data/labels/val/')
move_files_to_folder(test_annotations, 'split_data/labels/test/')

In [ ]:
!cp "{path}"best.onnx /content


In [ ]:
!python segment/predict.py --weights /content/best.onnx --data data.yaml --name yolo_segmentation --source split_data/images/val/ --line-thickness 1 --save-txt --save-masks #--nosave-with-bbox